In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from absbox import Generic,API,EnginePath,readBondsCf

In [3]:
localAPI = API(EnginePath.LOCAL,check=False,lang='english')

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.28.10

In [4]:
myAsset = ["ProjectedFlowFix"
          ,{"flows":[["2024-09-01",100,10,10]
                    ,["2024-12-01",90,10,10]
                    ,["2025-03-01",80,10,10]
                    ]
           ,"beginDate":"2024-06-01"
           ,"beginBalance":110}
          ,"MonthEnd"]

r = localAPI.runAsset("2024-08-02"
                     ,[myAsset]
                     ,poolAssump=("Pool",("Mortgage",{'CDR':0.01},{"CPR":0.02},None,None)
                                        ,None
                                        ,None)
                     ,read=True)

In [5]:
r[0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-09-01,99.26,9.92,9.92,0.55,0.27,0,0.27,0,None,None,9.92,0.55,0,0.27,0,0.27
2024-12-01,88.69,9.84,9.84,0.49,0.24,0,0.24,0,None,None,19.76,1.04,0,0.51,0,0.51
2025-03-01,78.28,9.77,9.77,0.43,0.21,0,0.21,0,None,None,29.53,1.47,0,0.72,0,0.72


### floater rate

In [6]:
myAsset = ["ProjectedFlowMix"
          ,{"flows":[["2024-09-01",100,10,10]
                    ,["2024-12-01",90,10,10]
                    ,["2025-03-01",80,10,10]
                    ]
           ,"beginDate":"2024-06-01"
           ,"beginBalance":110}
          ,"MonthEnd"
          ,(.5,0.08)
          ,[(1.0,0.02,"LIBOR1M")]]

r = localAPI.runAsset("2024-08-02"
                     ,[myAsset]
                     ,poolAssump=("Pool",("Mortgage",{'CDR':0.01},{"CPR":0.02},None,None)
                                        ,None
                                        ,None)
                     ,rateAssump=[("LIBOR1M",0.15)]
                     ,read=True)
r[0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-09-01,99.64,9.96,13.71,0.27,0.13,0,0.13,0,None,None,9.96,0.27,0,0.13,0,0.13
2024-12-01,89.03,9.88,12.36,0.49,0.24,0,0.24,0,None,None,19.84,0.76,0,0.37,0,0.37
2025-03-01,78.59,9.80,11.04,0.43,0.21,0,0.21,0,None,None,29.64,1.19,0,0.58,0,0.58


In [7]:
myAsset = ["ProjectedFlowMix"
          ,{"flows":[["2024-09-01",100,10,10]
                    ,["2024-12-01",90,10,10]
                    ,["2025-03-01",80,10,10]
                    ]
           ,"beginDate":"2024-06-01"
           ,"beginBalance":110}
          ,"MonthEnd"
          ,(.6,0.08)
          ,[(0.5,0.02,"LIBOR1M")
            ,(0.5,0.02,"LIBOR3M")]]

r = localAPI.runAsset("2024-08-02"
                     ,[myAsset]
                     ,poolAssump=("Pool",("Mortgage",{'CDR':0.01},{"CPR":0.02},None,None)
                                        ,None
                                        ,None)
                     ,rateAssump=[("LIBOR1M",0.30),("LIBOR3M",0.010)]
                     ,read=True)
r[0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-09-01,99.56,9.95,12.93,0.33,0.16,0,0.16,0,None,None,9.95,0.33,0,0.16,0,0.16
2024-12-01,88.97,9.86,11.66,0.49,0.24,0,0.24,0,None,None,19.81,0.82,0,0.40,0,0.40
2025-03-01,78.54,9.80,10.42,0.42,0.21,0,0.21,0,None,None,29.61,1.24,0,0.61,0,0.61


### Pool Run

In [20]:
myPool = {'assets':[ myAsset ],
         'cutoffDate':"2024-06-01"}

r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},{"CPR":0.02},None,None)
                                    ,None
                                    ,None)
                  ,rateAssump=[["LIBOR1M",0.30],["LIBOR3M",0.010]]   
                  ,read=True)
r['PoolConsol'][0]

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-09-01,99.56,9.95,12.93,0.33,0.16,0,0.16,0,None,None,9.95,0.33,0,0.16,0,0.16
2024-12-01,88.97,9.86,11.66,0.49,0.24,0,0.24,0,None,None,19.81,0.82,0,0.40,0,0.40
2025-03-01,78.54,9.80,10.42,0.42,0.21,0,0.21,0,None,None,29.61,1.24,0,0.61,0,0.61


### Deal Run

In [29]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[ myAsset ],'cutoffDate':"2024-06-01"}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [33]:
r = localAPI.run(test01
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},{"CPR":0.02},None,None)
                                    ,None
                                    ,None)
                  ,runAssump=[("interest",["LIBOR1M",0.30],["LIBOR3M",0.010])]   
                  ,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off

In [34]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-06-15,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-06-30,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-07-31,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-08-31,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-09-30,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-10-31,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-11-30,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2021-12-31,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
2022-01-31,110.00,0.00,0.00,0.00,0.00,0,0.00,0,None,None,0.00,0.00,0,0.00,0,0.00
